In [2]:
import glaredb
import polars as pl
import numpy as np
import os


In [ ]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


con = glaredb.connect("./db")
con.execute(f"create credential openai_creds provider openai OPTIONS (api_key = '{OPENAI_API_KEY}');") 
con.execute("""
    CREATE TABLE embedded_functions as (
        SELECT
            openai_embed(@creds.openai.openai_creds, 'text-embedding-3-small', concat_ws(';', function_name, example, description)) as embeddings, 
            * 
        FROM glare_catalog.functions
    );
""")
res = con.sql("SELECT * FROM embedded_functions").to_polars()

res

In [ ]:
res = con.sql("""
  SELECT cosine_similarity(
  openai_embed(
    @creds.openai.openai_creds, 
    'text-embedding-3-small', 
    'functions that work on string datatypes'
  ),
  embeddings
) 
FROM embedded_functions;
""").to_polars()